# Aula 4 - Parte prática - Actor-Critic

## Introdução 

Bla bla bla

In [ ]:
import logging

import gym
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp

from utils.agent import RLAgent
from utils.memory import MyReplayClass
from utils.networks import build_discrete_policy, build_value_network
import utils.runner
from utils.viz import *


tf.get_logger().setLevel("ERROR")     # ignore TensorFlow warnings
gym.logger.set_level(logging.ERROR)   # ignore OpenAI Gym warnings

In [ ]:
class A2C(RLAgent):
    
    def __init__(self, obs_space, action_space, config):
        super().__init__(obs_space, action_space, config)
        
        self.memory = MyReplayClass()
        self.policy = build_discrete_policy(self.obs_space, self.action_space, config["hidden_layers"], config["activation"])
        vf_config = config["value_fn"]
        self.value_fn = build_value_network(obs_space, vf_config["hidden_layers"], vf_config["activation"])

        self.optimizer = tf.keras.optimizers.get(config["optimizer"])
    
    def act(self, obs):
        """
        Escolhe uma ação para ser tomada dada uma observação do ambiente.
        
        Args: 
            obs: observação do ambiente.
        
        Return:
            action: ação válida dentro do espaço de ações.
        """
        return self._act(obs).numpy()
    
    @tf.function
    def _act(self, obs):
        action_dist = self.policy(obs)
        return action_dist.sample()

    def observe(self, obs, action, reward, next_obs, done):
        """
        Registra na memória do agente uma transição do ambiente.

        Args:
            obs:            observação do ambiente antes da execução da ação.
            action:         ação escolhida pelo agente.
            reward (float): escalar indicando a recompensa obtida após a execução da ação.
            next_obs:       nova observação recebida do ambiente após a execução da ação.
            done (bool):    True se a nova observação corresponde a um estado terminal, False caso contrário.

        Return:
            None
        """
        self.memory.update(obs, action, reward, next_obs, done)

    def learn(self):
        """
        Método de treinamento do agente. A partir das experiências de sua memória,
        o agente aprende um novo comportamento.

        Args: 
            None

        Return:
            None
        """
        if self.memory.batch_size < self.config["train_batch_size"]:
            return
        
        batch = self.memory.sample()
        weights = self.policy.trainable_weights + self.value_fn.trainable_weights

        with tf.GradientTape() as tape:
            loss = self._joint_loss_fn(batch)
            gradients = tape.gradient(loss, weights)

        self.optimizer.apply_gradients(zip(gradients, weights))
      
        return loss

    def _joint_loss_fn(self, batch):
        """
        Calcula a função loss do policy gradients para um `batch` de transições.
        
        Um `batch` agrega arrays n-dimensionais. Cada array (e.g., batch["states"],
        batch["actions"], batch["rewards"]) tem como primeiras duas dimensões o número
        de passos dados no ambiente vetorizado e o número de ambientes em paralelo. 
        Por exemplo, batch["states"][t][k] devolve um array correspondendo ao estado 
        no passo t devolvido pelo k-ésimo ambiente.

        Args:
            batch (Dict[str, np.ndarray]): dicionário para acesso às matrizes de 
                estados, ações, recompensas, próximos estados e flags de terminação. 
        
        Return:
            loss (tf.Tensor): surrogate loss conjunta da política, função valor e
                bônus de entropia.
        """
        vf_criterion = tf.keras.losses.MeanSquaredError()

        states = batch["states"]
        actions = batch["actions"]
        rewards = batch["rewards"]
        next_states = batch["next_states"]
        dones = batch["dones"]

        n_steps = len(states)
        gamma = self.config["gamma"]
        lam = self.config["lambda"]
        
        values = self.value_fn(states)
        next_values = self.value_fn(next_values)
        deltas = rewards + gamma * (1 - dones) * next_values - values

        returns = np.empty_like(rewards)
        advantages = np.empty_like(rewards)
        returns[-1] = rewards[-1] + gamma * (1 - dones) * next_values
        advantages[-1] = deltas[-1]
        for t in reversed(range(n_steps - 1)):
            returns[t] = rewards[t] + gamma * (1 - dones[t]) * returns[t+1]
            advantages[t] = deltas[t] + (gamma * lam) * (1 - dones[t]) * advantages[t+1]
            
        action_dists = self.policy(states)
        log_probs = action_dists.log_prob(actions)
        policy_loss = - tf.reduce_sum(log_probs * tf.stop_gradient(advantages))
        vf_loss = vf_criterion(values, tf.stop_gradient(returns))
        entropy_loss = tf.reduce_mean(action_dists.entropy())

        return policy_loss + self.config["vf_loss_coeff"] * vf_loss - self.config["entropy_coeff"] * entropy_loss